In [153]:
import sqlite3, datetime
from openpyxl import load_workbook

In [154]:
def getEntry(sheet, r):
    dat = sheet.cell(row = r, column = 1).value
    if not (dat is None or isinstance(dat, datetime.datetime)):
        return None
    pidCell = sheet.cell(row = r, column = 2)
    pid = pidCell.value
    if pidCell.data_type != 's' or pid is None:
        return None
    amountCell = sheet.cell(row = r, column = 3)
    amount = amountCell.value
    if amountCell.data_type != 'n' or amount is None:
        return None
    priceCell = sheet.cell(row = r, column = 4)
    price = priceCell.value
    if priceCell.data_type != 'n' or price is None:
        return None
    return (r, dat, pid, amount, price)

def rowAsString(sheet, r, highest_column):
    cells = [sheet.cell(row = r, column = i) for i in xrange(1, 1+highest_column)]
    return " ".join([c.value.strip() for c in cells if c.data_type == 's'])

def readSheet(sheet):
    title = None
    entries = []
    highest_row = sheet.get_highest_row()
    highest_column = sheet.get_highest_column()
    entry = getEntry(sheet, 1)
    if entry is None:
        title = rowAsString(sheet, 1, highest_column)
        if not title:
            title = None
    else:
        entries.append(entry)
    for r in xrange(2, 1+highest_row):
        entry = getEntry(sheet, r)
        if entry is not None:
            entries.append(entry)
    return (title, entries)

def readSheetFile(fname):
    wb = load_workbook(fname)
    sheetNames = wb.get_sheet_names()
    sheet = wb[sheetNames[0]]
    return readSheet(sheet)

In [155]:
def createTables(conn):
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS sheets(
                    sheetId INTEGER PRIMARY KEY AUTOINCREMENT,
                    flag INTEGER,
                    loadDate DATETIME,
                    title STRING,
                    filename STRING,
                    desc STRING
                    )''')
                    
    c.execute('''CREATE TABLE IF NOT EXISTS tranx(
                    sheetId INTEGER,
                    lineNo INTEGER,
                    date DATETIME,
                    productId STRING,
                    amount STRING,
                    price DOUBLE,
                    PRIMARY KEY(sheetId, lineNo)
                    )''')
    
    c.execute('''CREATE TABLE IF NOT EXISTS commands(
                    commandId INTEGER PRIMARY KEY AUTOINCREMENT,
                    flag INTEGER,
                    execDate DATETIME,
                    endDate DATETIME,
                    command INTEGER,
                    sheetId INTEGER,
                    desc STRING
                    )''')
    
    c.execute('''CREATE VIEW IF NOT EXISTS productTranx AS
                    SELECT productId, amount, price, date, tranx.sheetId, lineNo
                    FROM tranx JOIN sheets ON tranx.sheetId = sheets.sheetId WHERE sheets.flag=1
                    ORDER BY productId
                ''')
    
    c.execute('''CREATE VIEW IF NOT EXISTS productBalance AS
                    SELECT productId, SUM(amount) AS balance
                    FROM productTranx
                    GROUP BY productId
                ''')

    conn.commit()

In [156]:
conn = sqlite3.connect('inven.sqlite')
createTables(conn)

In [157]:
def loadSheetContent(conn, fname, title, entries):
    c = conn.cursor()
    c.execute('INSERT INTO commands(execDate, command) VALUES(CURRENT_TIMESTAMP, 1)')
    commandId = c.lastrowid
    
    c.execute('INSERT INTO sheets(title, filename) VALUES(:t, :f)', {'t': title, 'f': fname})
    sheetId = c.lastrowid
    
    cmd = 'INSERT INTO tranx(sheetId, lineNo, date, productId, amount, price) VALUES(%d, ?, ?, ?, ?, ?)' % sheetId
    #print cmd
    c.executemany(cmd, entries)
    
    c.execute('UPDATE sheets SET flag=1, loadDate=CURRENT_TIMESTAMP WHERE sheetId=:s', {'s': sheetId})
    c.execute('UPDATE commands SET flag=1, endDate=CURRENT_TIMESTAMP, sheetId=:s WHERE commandId=:c',
              {'s': sheetId, 'c': commandId})
    
    conn.commit()

def loadSheetFile(conn, fname):
    title, entries = readSheetFile(fname)
    loadSheetContent(conn, fname, title, entries)

In [158]:
loadSheetFile(conn, 'data1.xlsx')
loadSheetFile(conn, 'data2.xlsx')